# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [2]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/konstantinvaliotti/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/konstantinvaliotti/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [4]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [5]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [6]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [7]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/Users/konstantinvaliotti/AIE7_dev/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/Users/konstantinvaliotti/AIE7_dev/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/Users/konstantinvaliotti/AIE7_dev/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [8]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [9]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [24]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '08af42'. Skipping!
Property 'summary' already exists in node 'af9e8d'. Skipping!
Property 'summary' already exists in node 'd32f1e'. Skipping!
Property 'summary' already exists in node 'a66faf'. Skipping!
Property 'summary' already exists in node '4c48b1'. Skipping!
Property 'summary' already exists in node 'bfbde1'. Skipping!
Property 'summary' already exists in node '4be3bd'. Skipping!
Property 'summary' already exists in node 'fe3b2a'. Skipping!
Property 'summary' already exists in node 'c0a10e'. Skipping!
Property 'summary' already exists in node 'ef50c6'. Skipping!
Property 'summary' already exists in node 'd11f57'. Skipping!
Property 'summary' already exists in node 'c4381b'. Skipping!
Property 'summary' already exists in node '31366f'. Skipping!
Property 'summary' already exists in node '2fd6e9'. Skipping!


Applying CustomNodeFilter:   0%|          | 0/6 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/43 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node 'af9e8d'. Skipping!
Property 'summary_embedding' already exists in node 'c0a10e'. Skipping!
Property 'summary_embedding' already exists in node 'c4381b'. Skipping!
Property 'summary_embedding' already exists in node 'd11f57'. Skipping!
Property 'summary_embedding' already exists in node 'fe3b2a'. Skipping!
Property 'summary_embedding' already exists in node 'a66faf'. Skipping!
Property 'summary_embedding' already exists in node '4c48b1'. Skipping!
Property 'summary_embedding' already exists in node '4be3bd'. Skipping!
Property 'summary_embedding' already exists in node 'd32f1e'. Skipping!
Property 'summary_embedding' already exists in node '2fd6e9'. Skipping!
Property 'summary_embedding' already exists in node 'ef50c6'. Skipping!
Property 'summary_embedding' already exists in node '08af42'. Skipping!
Property 'summary_embedding' already exists in node 'bfbde1'. Skipping!
Property 'summary_embedding' already exists in node '31366f'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 40, relationships: 480)

We can save and load our knowledge graphs as follows.

In [25]:
kg.save("loan_data_kg.json")
loan_data_kg = KnowledgeGraph.load("loan_data_kg.json")
loan_data_kg

KnowledgeGraph(nodes: 40, relationships: 480)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [26]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loan_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [27]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


##### ✅ Answer:
1. SingleHopSpecific: A **single** retrieval is enough to answer this query. The question has **specific, fact-based** answer. Example: "Who are the authors of %paperName%?" -> Retrieval for QLoRA paper authors -> "The authors are ..." 
2. MultiHopAbstract: **Multiple** retrievals (>=2 sources/documents) are needed to answer this query with system needing to synthesize information from these sources. The question requires **interpretive response (explanation, interpretation, conclusion, etc.)** Example: "What approaches to finetuning are the most beneficial and how did these approaches evolve?" -> Retrievals about approaches (1), their benefits (2), each specific approach (i) -> "The most beneficial depends on...; For X: ...; For Y: ..."
3. MultiHopSpecific: **Multiple** retrievals (>=2 sources/documents) are needed to answer this query with system needing to synthesize information from these sources. The question has **specific, fact-based** answer (or answers). Example: "How does QLoRA work, what other finetuning approaches exist?" -> Retrievals related QLoRA (1), finetuning approaches (2) -> "QLoRA ...; other finetuning approaches are ..."
4. Bonus. SingleHopAbstract: A **single** retrieval is enough to answer this query. The question requires **interpretive response (explanation, interpretation, conclusion, etc.)** Example: "What makes QLoRA great?" -> Retrieval for QLoRA benefits -> "QLoRA benefits are: ..."

Finally, we can use our `TestSetGenerator` to generate our testset!

In [28]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,Whaat is the role of the School Participaation...,"[Chapter 1 Academic Years, Academic Calendars,...",The context does not provide specific details ...,single_hop_specifc_query_synthesizer
1,What is 34 CFR 668.3(a)?,[Regulatory Citations Academic year minimums: ...,34 CFR 668.3(a) is the regulatory citation for...,single_hop_specifc_query_synthesizer
2,What is Volume 8 in relation to clinical work ...,[Inclusion of Clinical Work in a Standard Term...,Volume 8 includes guidance on clinical work in...,single_hop_specifc_query_synthesizer
3,How does the Federal Work-Study program differ...,[Non-Term Characteristics A program that measu...,The Federal Work-Study (FWS) program is an exc...,single_hop_specifc_query_synthesizer
4,What is Volume 8 about?,[both the credit or clock hours and the weeks ...,Volume 8 discusses the effect of accelerated p...,single_hop_specifc_query_synthesizer
5,how payment periods and installment scheduling...,[<1-hop>\n\nboth the credit or clock hours and...,The context explains that disbursement of fede...,multi_hop_abstract_query_synthesizer
6,nonstandard terms and payment periods how they...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The context explains that clinical work outsid...,multi_hop_abstract_query_synthesizer
7,what r regulations and legal requirments for i...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",the regulations in 34 CFR 668.3(a) specify tha...,multi_hop_abstract_query_synthesizer
8,Appendix A and B how disbursement timing and c...,[<1-hop>\n\nboth the credit or clock hours and...,Appendix A explains that in clock-hour or non-...,multi_hop_specific_query_synthesizer
9,How do the definitions of academic years in Vo...,[<1-hop>\n\nboth the credit or clock hours and...,Volume 2 explains that an academic year must i...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [29]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '8d2ff3'. Skipping!
Property 'summary' already exists in node 'b5c7c3'. Skipping!
Property 'summary' already exists in node 'bf00b1'. Skipping!
Property 'summary' already exists in node 'f1f0e3'. Skipping!
Property 'summary' already exists in node '8f3e6a'. Skipping!
Property 'summary' already exists in node '57162f'. Skipping!
Property 'summary' already exists in node 'bf4acf'. Skipping!
Property 'summary' already exists in node '3c946a'. Skipping!
Property 'summary' already exists in node '17ec1d'. Skipping!
Property 'summary' already exists in node '395e79'. Skipping!
Property 'summary' already exists in node 'ddde7f'. Skipping!
Property 'summary' already exists in node '9da82c'. Skipping!
Property 'summary' already exists in node 'd5c780'. Skipping!
Property 'summary' already exists in node 'dbb6a7'. Skipping!


...

Property 'summary_embedding' already exists in node '8d2ff3'. Skipping!
Property 'summary_embedding' already exists in node '8f3e6a'. Skipping!
Property 'summary_embedding' already exists in node '395e79'. Skipping!
Property 'summary_embedding' already exists in node 'b5c7c3'. Skipping!
Property 'summary_embedding' already exists in node 'dbb6a7'. Skipping!
Property 'summary_embedding' already exists in node '3c946a'. Skipping!
Property 'summary_embedding' already exists in node 'bf00b1'. Skipping!
Property 'summary_embedding' already exists in node '57162f'. Skipping!
Property 'summary_embedding' already exists in node '9da82c'. Skipping!
Property 'summary_embedding' already exists in node 'bf4acf'. Skipping!
Property 'summary_embedding' already exists in node 'd5c780'. Skipping!
Property 'summary_embedding' already exists in node 'f1f0e3'. Skipping!
Property 'summary_embedding' already exists in node '17ec1d'. Skipping!
Property 'summary_embedding' already exists in node 'ddde7f'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [30]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What is Knowledge Center?,"[Chapter 1 Academic Years, Academic Calendars,...",The context does not provide a specific defini...,single_hop_specifc_query_synthesizer
1,What is the regulation 34 CFR 668.3(b) about i...,[Regulatory Citations Academic year minimums: ...,Regulatory Citations Academic year minimums: 3...,single_hop_specifc_query_synthesizer
2,What is the significance of Chapter 3 in relat...,[Inclusion of Clinical Work in a Standard Term...,Inclusion of clinical work in a standard term ...,single_hop_specifc_query_synthesizer
3,How does the Federal Work-Study (FWS) program ...,[Non-Term Characteristics A program that measu...,The payment period is applicable to all Title ...,single_hop_specifc_query_synthesizer
4,Participation and requirements for clinical or...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The inclusion of clinical or practicum experie...,multi_hop_abstract_query_synthesizer
5,do participation and reqmts for clinical or pr...,[<1-hop>\n\nInclusion of Clinical Work in a St...,"Yes, inclusion of clinical work in standard te...",multi_hop_abstract_query_synthesizer
6,How does the proration of loan limits for acce...,[<1-hop>\n\nboth the credit or clock hours and...,The proration of loan limits for accelerated s...,multi_hop_abstract_query_synthesizer
7,How do the academic calendar requirements outl...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The context explains that each eligible progra...,multi_hop_abstract_query_synthesizer
8,How do Appendix A and Appendix B relate to dis...,[<1-hop>\n\nDisbursement Timing in Subscriptio...,Appendix B provides detailed guidance on disbu...,multi_hop_specific_query_synthesizer
9,How does Volume 8 relate to the inclusion of c...,[<1-hop>\n\nInclusion of Clinical Work in a St...,Volume 8 provides guidance on the inclusion of...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [31]:
from langsmith import Client

client = Client()

dataset_name = "Loan Synthetic Data"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Loan Synthetic Data"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [32]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [33]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [34]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [35]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [36]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan RAG"
)

In [37]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [38]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

##### ✅ COMMENT:
The paragraph above seems to come from a different notebook / wasn't adapted to this course. Best update for next cohort! 

In [39]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [40]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [41]:
rag_chain.invoke({"question" : "What kinds of loans are available?"})

'The kinds of loans available mentioned in the context are:\n\n- Direct Subsidized Loan  \n- Direct Unsubsidized Loan  \n- Direct PLUS Loan (including student Direct PLUS Loans and parent PLUS Loans)  \n- Subsidized and Unsubsidized Federal Stafford Loans  \n- Federal SLS Loans  \n- Federal PLUS Loans  \n\nNote: Subsidized and Unsubsidized Federal Stafford Loans, Federal SLS Loans, and Federal PLUS Loans were made under the Federal Family Education Loan (FFEL) Program before the authority to make new FFEL Program loans ended effective July 1, 2010. New loans under these types are no longer made.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [42]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [43]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `empathy_evaluator`:

##### ✅ Answer:
1. qa_evaluator: "Question answering evaluator". LLM-Judge evaluates whether evaluated answer matches the correct answer.
2. labeled_helpfulness_evaluator: LLM-Judge evaluates how helpful the model's answer is based on the reference labels (used as criteria). In other words, evaluator knows the model's answer and the correct answer (labeled criteria), and using that knowledge evaluates custom-defined "helpfulness" based on that knowledge and our prompt. 
3. empathy_evaluator: LLM-Judge evaluate how empathetic the model's answer is. Evaluator doesn't know the correct answer and only evaluate the custom-defined "empathy" based on our prompt. 

## LangSmith Evaluation

In [44]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'reflecting-head-54' at:
https://smith.langchain.com/o/190538f0-9737-4132-9112-5b0de958d02b/datasets/b2a062c5-ada2-4989-8112-ff387d983194/compare?selectedSessions=9a45d02d-e93f-402b-863d-a1abb872afbf




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,Volume 2 and Volume 8 include info about clini...,I don't know.,None,Volume 8 discusses exceptions for clinical wor...,0,0,0,1.834888,b2eecb7b-dfab-4215-8a11-3947d15d886a,09e64f98-7c7c-4f32-995e-d054bcda09d9
1,How do Volume 2 and Volume 8 relate to academi...,I don't know.,None,Volume 2 discusses the academic year requireme...,0,0,0,0.968634,28107834-9248-4b81-89c7-e9d654cf5007,13deb9e9-4ac4-44e4-8faf-9122b4aa8f67
2,How does Volume 8 relate to the inclusion of c...,"Based on the provided context, Volume 8 addres...",None,Volume 8 provides guidance on the inclusion of...,1,1,0,4.703490,8ba01e75-b1a5-4172-8dfc-83a67fa9c60b,23b122f1-33cc-4ce7-a508-f47f3e38e52f
3,How do Appendix A and Appendix B relate to dis...,"Based on the provided context, Appendix B spec...",None,Appendix B provides detailed guidance on disbu...,0,0,0,3.583592,d5dda74b-d3f0-45bb-925f-b6628c26fc13,dc5c671b-12eb-46f0-9cd7-abdf9627a089
4,How do the academic calendar requirements outl...,The academic calendar requirements relate to t...,None,The context explains that each eligible progra...,1,1,0,3.673045,3cabcf2c-6750-4add-844a-46da8db1a80b,6bfa65fd-5c90-4458-afe3-c1e4595ae942
5,How does the proration of loan limits for acce...,I don't know.,None,The proration of loan limits for accelerated s...,0,0,0,1.123630,d1c583b1-e351-413f-b03f-d5a4a0189ff5,e78ba2c1-6b3a-46dd-84dd-701fd124e8ae
6,do participation and reqmts for clinical or pr...,"Based on the provided context, participation a...",None,"Yes, inclusion of clinical work in standard te...",0,1,0,4.130295,db7068ce-a289-4f38-9d7e-a6d17471466e,a8f924d8-aee9-4e8b-ad7e-b4fe48f2a8ef
7,Participation and requirements for clinical or...,Based on the provided context:\n\n- Clinical o...,None,The inclusion of clinical or practicum experie...,1,1,0,7.291885,6cf4a0e1-14da-42c5-b10f-7e2c47772ed8,3389dc74-c9bf-476c-a4e8-a9a074f0a9d2
8,How does the Federal Work-Study (FWS) program ...,The Federal Work-Study (FWS) program differs f...,None,The payment period is applicable to all Title ...,1,1,0,5.993512,823647e5-de27-436c-aad9-7317baa6c88c,9edc3859-cb1c-477b-9325-7e574849dd83
9,What is the significance of Chapter 3 in relat...,Chapter 3 in Volume 8 provides additional guid...,None,Inclusion of clinical work in a standard term ...,1,0,0,2.066482,8e7f1488-8316-42bf-a5ad-cb97ef6cc775,87888601-1370-4228-a615-06588581b845


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [45]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [46]:
rag_documents = docs

In [47]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

##### ✅ Answer:
Bigger chunk size automatically means more document context for the model from retrieval provided top_k stays the same. For example, with chunk_size 500 and top_k 5, the model will get up to 2500 total tokens (500x5). If we change chunk_size to 1000, the model will get up to 5000 total tokens (1000x5). While bigger context is no guarantee for better context, we eliminate a potential issues with not having enough info in it. 

P.S. In extremely large data sets significantly bigger chunk sizes may lead to 'averaged' embeddings in chunks, which makes each chunk less sensitive to highly specific concept retrieval. However, I think it is rather a theorethical problem. 

In [48]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

##### ✅ Answer:
By modifying the embedding model from small to large, we changed the number of embedding dimensions. Embeddings are numerical representations of concepts. Higher number of embedding dimensions means that they will capture more nuanced and specific information. This means that our retrieval queries will get us more relevant results. 

P.S. We can have "too many" embedding dimensions. This happens when there are more dimensions than meaningful conceptual similarity/dissimilarity signals, and the embedding model has to start capturing noise. This is, however, a theorethical problem I'll probably never face. 


In [49]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan Data for RAG"
)

In [50]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [51]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [52]:
empathy_rag_chain.invoke({"question" : "What kinds of loans are available?"})

"Thank you for your question—it's clear you're seeking to understand the types of loans available, which is an important step in navigating financial aid. Based on the information provided, there are several kinds of federal student loans mentioned:\n\n1. **Direct Subsidized Loans** – These loans are available based on the student’s financial need, covering the cost of attendance minus other aid received. The government pays the interest while the student is in school.\n\n2. **Direct Unsubsidized Loans** – These are available to students regardless of financial need and can be used in combination with subsidized loans or to cover unmet financial need.\n\n3. **Direct PLUS Loans** – These loans can be taken out by parents of dependent students or by independent students themselves. They can cover the student’s entire cost of attendance, minus other aid, assuming the borrower qualifies.\n\nAlso, if a parent is unable to obtain a Direct PLUS Loan, dependent students may be eligible for add

Finally, we can evaluate the new chain on the same test set!

In [53]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'cooked-humor-30' at:
https://smith.langchain.com/o/190538f0-9737-4132-9112-5b0de958d02b/datasets/b2a062c5-ada2-4989-8112-ff387d983194/compare?selectedSessions=bf23c792-86ef-4a56-ae31-f2afb51c772c




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,Volume 2 and Volume 8 include info about clini...,Thank you for your thoughtful question—it’s cl...,None,Volume 8 discusses exceptions for clinical wor...,1,1,1,4.587706,b2eecb7b-dfab-4215-8a11-3947d15d886a,fff2f990-758a-44d9-b588-76d7a9160dd6
1,How do Volume 2 and Volume 8 relate to academi...,Thank you for your thoughtful question. Based ...,None,Volume 2 discusses the academic year requireme...,0,0,1,4.684278,28107834-9248-4b81-89c7-e9d654cf5007,663e6c47-edb6-41fe-b1f8-7966dd6b979b
2,How does Volume 8 relate to the inclusion of c...,Thank you for your thoughtful question. Based ...,None,Volume 8 provides guidance on the inclusion of...,1,0,1,2.682127,8ba01e75-b1a5-4172-8dfc-83a67fa9c60b,0af7ce89-d5ea-4e14-9d06-99c6acd806e0
3,How do Appendix A and Appendix B relate to dis...,Thank you for your thoughtful question. From t...,None,Appendix B provides detailed guidance on disbu...,0,0,1,3.545551,d5dda74b-d3f0-45bb-925f-b6628c26fc13,4f737381-d0ac-46b0-8bda-fa3744a05c7e
4,How do the academic calendar requirements outl...,Thank you for your thoughtful question. From t...,None,The context explains that each eligible progra...,1,0,1,4.221798,3cabcf2c-6750-4add-844a-46da8db1a80b,11919ea9-eecb-47c9-a7c1-74e714ce4c4e
5,How does the proration of loan limits for acce...,Thank you for your thoughtful question about h...,None,The proration of loan limits for accelerated s...,1,0,1,5.470932,d1c583b1-e351-413f-b03f-d5a4a0189ff5,1e323096-6b61-42fe-9d0f-b43a37e57747
6,do participation and reqmts for clinical or pr...,Thank you for your thoughtful question. Based ...,None,"Yes, inclusion of clinical work in standard te...",1,1,1,3.458647,db7068ce-a289-4f38-9d7e-a6d17471466e,9282bed6-c950-4ebf-9994-35a27067e8b6
7,Participation and requirements for clinical or...,Thank you for your thoughtful question—it touc...,None,The inclusion of clinical or practicum experie...,1,1,1,7.313238,6cf4a0e1-14da-42c5-b10f-7e2c47772ed8,d07c2aca-1025-4f49-9a47-cd7cd0340c6c
8,How does the Federal Work-Study (FWS) program ...,Thank you for your thoughtful question. Based ...,None,The payment period is applicable to all Title ...,1,1,1,4.340392,823647e5-de27-436c-aad9-7317baa6c88c,9a924381-74da-4723-8136-12d2b01a9f4c
9,What is the significance of Chapter 3 in relat...,Thank you for your thoughtful question. Based ...,None,Inclusion of clinical work in a standard term ...,1,1,1,3.872661,8e7f1488-8316-42bf-a5ad-cb97ef6cc775,0494266d-944d-4e97-9d6b-faac112d46f1


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

##### ✅ Answer:
1. Empathy jumped significantly: our new prompt instructs to provide empathethic responses while original prompt didn't include instruction for empathetic tone of voice.
2. Correctness improved: we increased embedding dimensions by using a larger model and increased chunk size. These changes led to new RAG pipeline having more in-context information with potentially more relevant information due to changes in embeddings. Manual review of logs indicates that there were: a. 2 cases where original answer was "I don't know" with the second run providing correct answers; b. 1 case where original answer was simply incorrect with the second run providign correct answer; c. 1 case where second run answer is marked as incorrect with first run answer marked as correct, but that's due to bad reference output (check p.4 and second screenshot).
3. Helpfulness didn't change. It is puzzling (given that correctness improved), so I manually expected both datasets. There are 2 cases where second run answers are marked as helpful with first run answers not being helpful, but there are also 2 cases where first run answers are marked as helpful with second run answers not being helpful. Case review showed that second run underperformed in some cases, providing fuzzy answers while first run was more on-point with providing information. (Screenshot 3 shows the example; original run provided information about both 30 and 26 weeks requirements, while second run only indicated 26 weeks; similar case is shown in screenshot 4) **The LLM in the second run may have overemphasized our instruction on making the user feel heard and being kind, which led to it treating user like an idiot, and overexplaining routine details without providing necessary information compared to the original run**
4. Manual review of logs (second screenshot) uncovered a funny case. LLM Evaluator provided different ratings for the same answer (correct for original run, incorrect for second run), however, this is due to reference output being low quality. 
![img](eval_diff.png)
![img](bad_reference_output.png)
![img](second_run_underperformance_case.png)
![img](second_run_another_underperformance_case.png)